## Analysis of Missing Pet Reports and Weather in Central Ohio

In [33]:
import pandas as pd
import duckdb

import plotly.express as px

import numpy as np
import statsmodels.api as sm

### Load and prepare weather data

In [34]:
weather_data = pd.read_csv('./data/GHCND_USW00004804.csv', index_col='DATE', parse_dates=True).loc[:, ['TMAX', 'PRCP']]
weekly_weather_data = weather_data.groupby(pd.Grouper(freq='7D')).mean()
weekly_weather_data['week'] = weekly_weather_data.index

### Load and prepare report data

In [35]:
daily_report_data = pd.read_csv('./data/daily_report_counts_central_oh.csv', index_col='event_date', parse_dates=True)
weekly_report_data = daily_report_data.groupby(pd.Grouper(freq='7D')).sum()
weekly_report_data['rolling_avg'] = weekly_report_data['reports'].rolling(5, center=True).mean()
weekly_report_data['week'] = weekly_report_data.index

### Load and prepare firework event data

In [36]:
firework_event_weeks = pd.read_csv('./data/firework_event_weeks.csv', index_col='event_week', parse_dates=True)
firework_event_weeks['week'] = firework_event_weeks.index

### Combine data and create features

In [37]:
weekly_summary = duckdb.query('''
  select
    r.week,
    cast(year(r.week) as varchar) as year,
    r.reports,
    case when f.is_event = 1 then r.rolling_avg
      else r.reports
    end as smoothed_reports,
    log10(r.reports) as log_reports,
    w.TMAX,
    w.PRCP,
    r.rolling_avg
  from weekly_report_data r
  join weekly_weather_data w
    on w.week = r.week
  left join firework_event_weeks f
    on f.week = r.week
  where r.rolling_avg is not null
    and year(r.week) between 2019 and 2022
  order by r.week
''').df()
weekly_summary.set_index('week')
weekly_summary.head()

,week,year,reports,smoothed_reports,log_reports,TMAX,PRCP,rolling_avg
0,2019-01-06,2019,130,130.0,2.113943,41.571429,0.027143,121.6
1,2019-01-13,2019,95,95.0,1.977724,32.142857,0.071429,111.2
2,2019-01-20,2019,96,96.0,1.982271,29.857143,0.077143,102.4
3,2019-01-27,2019,81,81.0,1.908485,26.142857,0.021429,100.0
4,2019-02-03,2019,110,110.0,2.041393,49.571429,0.270000,101.8


### Plot data distributions and trends

In [38]:
scatter_fig= px.scatter(
  x=weekly_summary['TMAX'],
  y=weekly_summary['smoothed_reports'],
  width=720,
  labels={
    'x': 'Averge Max Temperature',
    'y': 'Missing Pet Report Volume',
    'color': 'Year'
  },
  title='Weekly Missing Pet Report Volumes Compared to Average Temperature'
)
scatter_fig.show()

In [39]:
scatter_fig= px.scatter(
  x=weekly_summary['TMAX'],
  y=weekly_summary['smoothed_reports'],
  width=720,
  color=weekly_summary['year'],
  labels={
    'x': 'Averge Max Temperature',
    'y': 'Missing Pet Report Volume',
    'color': 'Year'
  },
  title='Weekly Missing Pet Report Volumes Compared to Average Temperature by Year'
)
scatter_fig.show()

In [40]:
line_fig = px.line(
  weekly_summary,
  x='week',
  y=['reports', 'TMAX', 'smoothed_reports', 'rolling_avg'],
  width=720,
  labels={
    'value': 'Value',
    'week': 'Week of Observation',
    'variable': 'Metric'
  },
  title='Weekly Missing Pet Report Volume and Max Temperature'
)
line_fig.show()

### Test for relationship with linear regression

In [41]:
ols_res = sm.OLS(weekly_summary['smoothed_reports'], sm.add_constant(weekly_summary[['TMAX', 'PRCP']])).fit()
print(ols_res.summary())

                            OLS Regression Results                            
Dep. Variable:       smoothed_reports   R-squared:                       0.273
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     38.49
Date:                Tue, 07 Mar 2023   Prob (F-statistic):           6.41e-15
Time:                        20:43:36   Log-Likelihood:                -1072.8
No. Observations:                 208   AIC:                             2152.
Df Residuals:                     205   BIC:                             2162.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        101.8629     10.766      9.462      0.0

In [42]:
ols_fig = px.line(
  weekly_summary,
  x=weekly_summary['TMAX'],
  y=ols_res.fittedvalues,
  width=720,
  labels={
    'x': 'Averge Max Temperature',
    'y': 'Missing Pet Report Volume'
  },
  title='Weekly Missing Pet Report Volumes Compared to Average Temperature',
  color_discrete_sequence=[px.colors.qualitative.T10[1]]
)
ols_fig.add_scatter(
  x=weekly_summary['TMAX'],
  y=weekly_summary['smoothed_reports'],
  mode='markers',
  marker={
    'color': px.colors.qualitative.T10[0]
  },
  name='Report Volume'
)
ols_fig.show()

In [43]:
ols_fig_resid = px.scatter(
  weekly_summary,
  x=weekly_summary['TMAX'],
  y=ols_res.resid,
  color='year',
  width=720,
  title='Residuals of Ordinary Least Squares Regressions Colored by Year',
  marginal_x='histogram',
  marginal_y='rug',
  color_discrete_sequence=px.colors.qualitative.T10
)
ols_fig_resid.show()

In [44]:
glm_res = sm.OLS(weekly_summary['smoothed_reports'], sm.add_constant(weekly_summary[['TMAX', 'year']].astype(float))).fit()
print(glm_res.summary())

                            OLS Regression Results                            
Dep. Variable:       smoothed_reports   R-squared:                       0.682
Model:                            OLS   Adj. R-squared:                  0.679
Method:                 Least Squares   F-statistic:                     220.3
Date:                Tue, 07 Mar 2023   Prob (F-statistic):           8.59e-52
Time:                        20:43:36   Log-Likelihood:                -986.66
No. Observations:                 208   AIC:                             1979.
Df Residuals:                     205   BIC:                             1989.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.696e+04   3508.398    -16.235      0.0

In [45]:
def mean_absolute_percentage_error(y, y_hat):
    y, y_hat = np.array(y), np.array(y_hat)
    return np.mean(np.abs((y - y_hat) / y))

In [46]:
print(sm.tools.eval_measures.meanabs(weekly_summary['smoothed_reports'], ols_res.fittedvalues))
print(sm.tools.eval_measures.rmspe(weekly_summary['smoothed_reports'], ols_res.fittedvalues))
print(mean_absolute_percentage_error(weekly_summary['smoothed_reports'], ols_res.fittedvalues))

36.47606061741912
2.4331179439541586
0.2028817296643918


In [47]:
glm_fig = px.line(
  weekly_summary,
  x=weekly_summary['TMAX'],
  y=glm_res.fittedvalues,
  width=720,
  labels={
    'x': 'Averge Max Temperature',
    'y': 'Missing Pet Report Volume',
    'year': 'Prediced Values'
  },
  title='Weekly Missing Pet Report Volumes Compared to Average Temperature',
  color='year',
  color_discrete_sequence=px.colors.qualitative.T10
)
for y in weekly_summary.year.unique():
  year_summary = weekly_summary[weekly_summary['year'] == y]
  glm_fig.add_scatter(
    x=year_summary['TMAX'],
    y=year_summary['smoothed_reports'],
    mode='markers',
    marker_color=px.colors.qualitative.T10[int(y) - 2019],
    name='Actual Volume ' + y
  )

glm_fig.show()

In [48]:
glm_fig_resid = px.scatter(
  weekly_summary,
  x=weekly_summary['TMAX'],
  y=glm_res.resid,
  color='year',
  width=720,
  title='Residuals of Ordinary Least Square Regression Colored by Year',
  marginal_x='histogram',
  marginal_y='rug',
  color_discrete_sequence=px.colors.qualitative.T10
)
glm_fig_resid.show()